# Market Basket Analysis

Market basket analysis scrutinizes the products customers tend to buy together, and uses the information to decide which products should be cross-sold or promoted together. The term arises from the shopping carts supermarket shoppers fill up during a shopping trip.

Association Rule Mining is used when we want to find an association between different objects in a set, find frequent patterns in a transaction database, relational databases or any other information repository.

The most common approach to find these patterns is Market Basket Analysis, which is a key technique used by large retailers like Amazon, Flipkart, etc to analyze customer buying habits by finding associations between the different items that customers place in their “shopping baskets”. The discovery of these associations can help retailers develop marketing strategies by gaining insight into which items are frequently purchased together by customers. The strategies may include:

- Changing the store layout according to trends
- Customers behavior analysis
- Catalog Design
- Cross marketing on online stores
- Customized emails with add-on sales, etc.

### Matrices

- **Support** : Its the default popularity of an item. In mathematical terms, the support of item A is the ratio of transactions involving A to the total number of transactions.


- **Confidence** : Likelihood that customer who bought both A and B. It is the ratio of the number of transactions involving both A and B and the number of transactions involving B.
     - Confidence(A => B) = Support(A, B)/Support(A)


- **Lift** : Increase in the sale of A when you sell B.
    
    - Lift(A => B) = Confidence(A, B)/Support(B)
        
    - Lift (A => B) = 1 means that there is no correlation within the itemset.
    - Lift (A => B) > 1 means that there is a positive correlation within the itemset, i.e., products in the itemset, A, and B, are more likely to be bought together.
    - Lift (A => B) < 1 means that there is a negative correlation within the itemset, i.e., products in itemset, A, and B, are unlikely to be bought together.

**Apriori Algorithm:** Apriori algorithm assumes that any subset of a frequent itemset must be frequent. Its the algorithm behind Market Basket Analysis. Say, a transaction containing {Grapes, Apple, Mango} also contains {Grapes, Mango}. So, according to the principle of Apriori, if {Grapes, Apple, Mango} is frequent, then {Grapes, Mango} must also be frequent.

In [1]:
!pip install presto-python-client
!pip install pandas plotnine
!pip install mlxtend

import prestodb
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

### Data

In [2]:
# limit the data to 1M rows for avoid OFME
conn=prestodb.dbapi.connect(
    host='presto-service',
    port=8282,
    user='presto',
    catalog='hive',
    schema='sample',
)

fetchlimit = 100000

cur = conn.cursor()
cur.execute('show tables')
pd.DataFrame.from_records(cur.fetchall(), columns = [i[0] for i in cur.description])



,Table
0,avg_product_count_by_dow
1,order_products
2,product_count
3,stg_order_products
4,stg_orders
5,top_selling_products


In [3]:
cur.execute('select * from sample.order_products limit ' + str(fetchlimit))
order_products = pd.DataFrame.from_records(cur.fetchall(), columns = [i[0] for i in cur.description])
order_products.shape

(100000, 21)

In [4]:
order_products.head()

,_hoodie_commit_time,_hoodie_commit_seqno,_hoodie_record_key,_hoodie_partition_path,_hoodie_file_name,order_id,user_id,order_number,order_dow,order_hour_of_day,...,days_since_prior_order_cum,order_date,product_id,add_to_cart_order,reordered,product_name,aisle_id,aisle,department_id,department
0,20220715145217724,20220715145217724_74_156134,"order_id:2140546,product_id:7361",,61cf5a4c-8127-4750-b285-a40163da5747-0_74-500-...,2140546,58491,25,6,15,...,103.0,2020-04-14 07:00:00.000,7361,4,True,Herb Crackers,78.0,crackers,19.0,snacks
1,20220715145217724,20220715145217724_74_156135,"order_id:2140546,product_id:7649",,61cf5a4c-8127-4750-b285-a40163da5747-0_74-500-...,2140546,58491,25,6,15,...,103.0,2020-04-14 07:00:00.000,7649,14,False,Whole Wheat Tandoori Naan,128.0,tortillas flat bread,3.0,bakery
2,20220715145217724,20220715145217724_74_156136,"order_id:2140575,product_id:13799",,61cf5a4c-8127-4750-b285-a40163da5747-0_74-500-...,2140575,143455,1,6,14,...,0.0,2020-01-07 06:00:00.000,13799,4,False,42.2 Oz Wj Rf Lav&Van,114.0,cleaning products,17.0,household
3,20220715145217724,20220715145217724_74_156137,"order_id:2140575,product_id:18477",,61cf5a4c-8127-4750-b285-a40163da5747-0_74-500-...,2140575,143455,1,6,14,...,0.0,2020-01-07 06:00:00.000,18477,8,False,Ground Round 85% Lean,122.0,meat counter,12.0,meat seafood
4,20220715145217724,20220715145217724_74_156138,"order_id:2140575,product_id:18610",,61cf5a4c-8127-4750-b285-a40163da5747-0_74-500-...,2140575,143455,1,6,14,...,0.0,2020-01-07 06:00:00.000,18610,1,False,Cola Soda,77.0,soft drinks,7.0,beverages


In [5]:
order_products.product_id.nunique()

15146

In [6]:
products = order_products.groupby(['product_id', 'product_name', 'aisle_id', 'department_id']).size().to_frame('order_count').reset_index()
products.head()

,product_id,product_name,aisle_id,department_id,count
0,1,Chocolate Sandwich Cookies,61.0,19.0,6
1,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38.0,1.0,1
2,9,Light Strawberry Blueberry Yogurt,120.0,16.0,1
3,10,Sparkling Orange Juice & Prickly Pear Beverage,115.0,7.0,9
4,16,Mint Chocolate Flavored Syrup,103.0,19.0,1


Out of 49685 keeping top 100 most frequent products.

In [7]:
product_counts = order_products.groupby('product_id')['order_id'].count().reset_index().rename(columns = {'order_id':'frequency'})
product_counts = product_counts.sort_values('frequency', ascending=False)[0:100].reset_index(drop = True)
product_counts = product_counts.merge(products, on = 'product_id', how = 'left')
product_counts.head(10)

,product_id,frequency,product_name,aisle_id,department_id,count
0,24852,1550,Banana,24.0,4.0,1550.0
1,13176,1179,Bag of Organic Bananas,24.0,4.0,1179.0
2,21137,768,Organic Strawberries,24.0,4.0,768.0
3,21903,763,Organic Baby Spinach,123.0,4.0,763.0
4,47209,650,Organic Hass Avocado,24.0,4.0,650.0
5,47766,594,Organic Avocado,24.0,4.0,594.0
6,47626,487,Large Lemon,24.0,4.0,487.0
7,26209,470,Limes,24.0,4.0,470.0
8,16797,427,Strawberries,24.0,4.0,427.0
9,27845,421,Organic Whole Milk,84.0,16.0,421.0


Keeping 100 most frequent items in order_products dataframe

In [8]:
freq_products = list(product_counts.product_id)
freq_products[1:10]

[13176, 21137, 21903, 47209, 47766, 47626, 26209, 16797, 27845]

In [9]:
len(freq_products)

100

In [10]:
order_products = order_products[order_products.product_id.isin(freq_products)]
order_products.shape

(23591, 21)

In [11]:
order_products.order_id.nunique()

7250

In [12]:
order_products.head()

,_hoodie_commit_time,_hoodie_commit_seqno,_hoodie_record_key,_hoodie_partition_path,_hoodie_file_name,order_id,user_id,order_number,order_dow,order_hour_of_day,...,days_since_prior_order_cum,order_date,product_id,add_to_cart_order,reordered,product_name,aisle_id,aisle,department_id,department
23,20220715145217724,20220715145217724_74_156157,"order_id:214065,product_id:21903",,61cf5a4c-8127-4750-b285-a40163da5747-0_74-500-...,214065,157347,3,5,15,...,19.0,2020-02-24 07:00:00.000,21903,5,False,Organic Baby Spinach,123.0,packaged vegetables fruits,4.0,produce
24,20220715145217724,20220715145217724_74_156158,"order_id:214065,product_id:22935",,61cf5a4c-8127-4750-b285-a40163da5747-0_74-500-...,214065,157347,3,5,15,...,19.0,2020-02-24 07:00:00.000,22935,4,False,Organic Yellow Onion,83.0,fresh vegetables,4.0,produce
37,20220715145217724,20220715145217724_74_156171,"order_id:2140653,product_id:5785",,61cf5a4c-8127-4750-b285-a40163da5747-0_74-500-...,2140653,186889,22,6,14,...,285.0,2020-11-16 06:00:00.000,5785,3,True,Organic Reduced Fat 2% Milk,84.0,milk,16.0,dairy eggs
41,20220715145217724,20220715145217724_74_156175,"order_id:2140777,product_id:21137",,61cf5a4c-8127-4750-b285-a40163da5747-0_74-500-...,2140777,142850,13,2,23,...,100.0,2020-04-10 15:00:00.000,21137,2,True,Organic Strawberries,24.0,fresh fruits,4.0,produce
53,20220715145217724,20220715145217724_74_156187,"order_id:2140828,product_id:21903",,61cf5a4c-8127-4750-b285-a40163da5747-0_74-500-...,2140828,190693,23,4,8,...,210.0,2020-08-02 00:00:00.000,21903,9,False,Organic Baby Spinach,123.0,packaged vegetables fruits,4.0,produce


Structuring the data for feeding in the algorithm

In [13]:
basket = order_products.groupby(['order_id', 'product_name'])['reordered'].count().unstack().reset_index().fillna(0).set_index('order_id')
basket.head()

product_name,"""Clementines",100% Raw Coconut Water,100% Whole Wheat Bread,2% Reduced Fat Milk,Apple Honeycrisp Organic,Asparagus,Bag of Organic Bananas,Banana,Bartlett Pears,Blueberries,...,Sparkling Lemon Water,Sparkling Natural Mineral Water,Sparkling Water Grapefruit,Spring Water,Strawberries,Uncured Genoa Salami,Unsalted Butter,Unsweetened Almondmilk,Unsweetened Original Almond Breeze Almond Milk,Yellow Onions
order_id,,,,,,,,,,,,,,,,,,,,,
359,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1804,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2849,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3655,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
del product_counts, products, order_products

NameError: name 'order_products_prior' is not defined

encoding the units

In [ ]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1 
    
basket = basket.applymap(encode_units)
basket.head()

In [ ]:
basket.size

In [ ]:
basket.shape

Creating frequent sets and rules

In [ ]:
frequent_items = apriori(basket, min_support=0.01, use_colnames=True, low_memory=True)
frequent_items.head()

In [ ]:
frequent_items.tail()

In [ ]:
frequent_items.shape

In [ ]:
rules = association_rules(frequent_items, metric="lift", min_threshold=1)
rules.sort_values('lift', ascending=False)

In [ ]:
conn.close()